## Inheritance

In [47]:
class Card():
    """Playing card"""
    
    rankMap = [("%d" % i) for i in range(13)] # Class attributes
    (rankMap[0],rankMap[1],*rankMap[11:13]) = ('None','Ace','Jack','Queen','King')
    # Interestingly, it seems that I can only use * in the left side once, so I cannot * both vectors
    # print(rankMap)
    suitMap = ['Clubs','Diamonds','Hearts','Spades']
    
    
    def __init__(self,suit=0,rank=1):
        self.suit = suit # Instance attributes
        self.rank = rank
        
    def __str__(self):
        return "%s of %s" % (Card.rankMap[self.rank] , Card.suitMap[self.suit])
           
    def __eq__(self,other):
        return (self.rank==other.rank) & (self.suit==other.suit)
    
    def __gt__(self,other): 
        # They removed __cmp__ in Python 3, replacing it with: eq, ne, lt, le, gt, ge
        # However, Python somehow guesses how > will behave knowing <. So the minimum includes EQ, GT & GE
        # And then there's also __repr__ that is used for eval(). In this case, will be "Card(0,1)" - but with actual values
        return (self.suit, self.rank) > (other.suit, other.rank) # Uses the fancy lexicographicish way tuples are compared
        
    def __ge__(self,other):
        return self.__gt__(other) | self.__eq__(other)
        

a = Card(0,2)
print(a)
print(a==Card(0,3))
print(a!=Card(0,3))
print(a>Card(0,3))
print(a<Card(0,3))
print(a<=Card(0,3))
print(a>=Card(0,3))

2 of Clubs
False
True
False
True
True
False
